Very simple implementation of the double pendulum. Equation taken from

https://web.mit.edu/jorloff/www/chaosTalk/double-pendulum/double-pendulum-en.html

Note, that I ignore any drift in total energy. To handle that one can rely on https://scipython.com/blog/the-double-pendulum/. The plotting is also based on that implementation.

In [ ]:
import sys
import numpy as np
from scipy.integrate import odeint
import matplotlib.pyplot as plt
from matplotlib.patches import Circle

# Pendulum rod lengths (au), masses (au).
L1, L2 = 1, 0.5
m1, m2 = 1, 1.0
g = 9.81

def deriv(y, t, L1, L2, m1, m2):
    """Implementing https://web.mit.edu/jorloff/www/chaosTalk/double-pendulum/double-pendulum-en.html"""
    theta1, theta2, omega1, omega2 = y
    
    theta1dot = omega1
    theta2dot = omega2
    
    omega1dot=(-g*(2*m1+m2)*np.sin(theta1)-m2*g*np.sin(theta1-2*theta2) -
              2*np.sin(theta1-theta2)*m2*(omega2**2*L2+omega1**2*L1*np.cos(theta1-theta2))) / \
             (L1*(2*m1+m2-m2*np.cos(2*omega1-2*omega2)))
    
    omega2dot=(2*np.sin(theta1-theta2)*(omega1**2*L1*(m1+m2)+g*(m1+m2)*np.cos(theta1) +
              omega2**2*L2*m2*np.cos(theta1-theta2))) / \
             (L2*(2*m1+m2-m2*np.cos(2*omega1-2*omega2)))
    return theta1dot, theta2dot, omega1dot, omega2dot

# Maximum time, time point spacings and the time grid (all in s).
tmax, dt = 30, 0.01
t = np.arange(0, tmax+dt, dt)

y0 = np.array([-np.pi/2, 0., np.pi/2, 0.])        #unperturbed
#y0 = np.array([-np.pi/2*1.0001, 0., np.pi/2, 0.]) #perturbed

# Do the numerical integration of the equations of motion
y = odeint(deriv, y0, t, args=(L1, L2, m1, m2))

theta1, theta2 = y[:,0], y[:,1]  #it is interesting to plot these against time, and observe when similar initial
                                 # conditions split in time.

# Convert to Cartesian coordinates of the masses, this will be then plotted.
x1 = L1 * np.sin(theta1)
y1 = -L1 * np.cos(theta1)
x2 = x1 + L2 * np.sin(theta2)
y2 = y1 - L2 * np.cos(theta2)

In [ ]:
r = 0.05
# Plot a trail of the m2 bob's position for the last trail_secs seconds.
trail_secs = 2
# This corresponds to max_trail time points.
max_trail = int(trail_secs / dt)

def make_plot(i):
    # Plot and save an image of the double pendulum configuration for time
    # point i.
    # The pendulum rods.
    ax.plot([0, x1[i], x2[i]], [0, y1[i], y2[i]], lw=2, c='gray')
    # Circles representing the anchor point of rod 1, and bobs 1 and 2.
    c0 = Circle((0, 0), r/2, fc='k', zorder=10)
    c1 = Circle((x1[i], y1[i]), r*m1, fc='k', ec='k', zorder=10)
    c2 = Circle((x2[i], y2[i]), r*m2, fc='k', ec='k', zorder=10)
    ax.add_patch(c0)
    ax.add_patch(c1)
    ax.add_patch(c2)

    # The trail will be divided into ns segments and plotted as a fading line.
    ns = 20
    s = max_trail // ns

    for j in range(ns):
        imin = i - (ns-j)*s
        if imin < 0:
            continue
        imax = imin + s + 1
        # The fading looks better if we square the fractional length along the
        # trail.
        alpha = (j/ns)**2
        ax.plot(x2[imin:imax], y2[imin:imax], c='k', solid_capstyle='butt',
                lw=2, alpha=alpha)

    # Centre the image on the fixed anchor point, and ensure the axes are equal
    ax.set_xlim(-L1-L2-r, L1+L2+r)
    ax.set_ylim(-L1-L2-r, L1+L2+r)
    ax.set_aspect('equal', adjustable='box')
    plt.axis('off')
    plt.savefig('pendulumframes1/_img{:04d}.png'.format(i//di), dpi=72)#,transparent=True) %Make the folder first!
    plt.cla()


# Make an image every di time points, corresponding to a frame rate of fps
# frames per second.
# Frame rate, s-1
fps = 10
di = int(1/fps/dt)
fig = plt.figure(figsize=(8.3333, 6.25))
ax = fig.add_subplot(111)

for i in range(0, t.size, di):
    print(i // di, '/', t.size // di)
    make_plot(i)